## Student: Yingtao, Zhang (V00943483)

# Problem 11


In [13]:
# This should be roughly the content of the first code cell
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)

In [14]:
# Plotting support
from matplotlib import pyplot as plt
# Standard libraries
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [15]:
df = pd.read_csv("fandango_score_comparison.csv")
df

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,...,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.30,...,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.00,...,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.50,...,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,0.90,4.20,...,2.70,1.0,4.0,1.0,2.5,2.5,31,3136,1793,0.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,0.70,1.40,...,2.55,0.5,1.5,1.5,1.5,2.5,88,19560,1021,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Mr. Holmes (2015),87,78,67,7.9,7.4,4.0,4.0,4.35,3.90,...,3.70,4.5,4.0,3.5,4.0,3.5,33,7367,1348,0.0
142,'71 (2015),97,82,83,7.5,7.2,3.5,3.5,4.85,4.10,...,3.60,5.0,4.0,4.0,4.0,3.5,60,24116,192,0.0
143,"Two Days, One Night (2014)",97,78,89,8.8,7.4,3.5,3.5,4.85,3.90,...,3.70,5.0,4.0,4.5,4.5,3.5,123,24345,118,0.0
144,Gett: The Trial of Viviane Amsalem (2015),100,81,90,7.3,7.8,3.5,3.5,5.00,4.05,...,3.90,5.0,4.0,4.5,3.5,4.0,19,1955,59,0.0


    The best features to answer this question: 
    Fandango_Ratingvalue
    Fandango_Difference

    By creating the testing column I found that 
    Fandango_Stars = Fandango_Ratingvalue + Fandango_Difference
    

In [16]:
best_features = ['Fandango_Ratingvalue','Fandango_Difference']

df["testing"] = df.apply((lambda x: (x.Fandango_Ratingvalue+x.Fandango_Difference) == x['Fandango_Stars']), axis =1)
# Uncomment the code below to see the test result
#print(df["testing"])


    Logistic Regression

\begin{array}{c}
\text { Sigmoid } \\
f(x)=\frac{1}{1+e^{\cdot x}}
\end{array}
$$

In [17]:
#Calculate the sigmoid funtion
def sigmoid(x):
    x = np.exp(-x)
    sigm = 1 / (1 + x)
    return sigm

$$
d W=(A-Y) \cdot X^{\top}
$$

In [18]:
#Get W gradient for each step
def get_W_gradient(A, X, y, num_ele):
    dW = (1/num_ele)*((A-y).dot(X.T))
    return dW

$$
d B=(A-Y)
$$


In [19]:
#Get B gradient for each step
def get_B_gradient(A, y, num_ele):
    dB = (1/num_ele)*np.sum(A - y)
    return dB


$$
\begin{array}{l}
R= \text {Learning Rate } \\
W=W -R \times d  W^{\top} \\
B=B-R \times d B
\end{array}
$$

In [20]:
#Get gradient for each step
def get_gradient(A, X, y, W, B, learning_rate, num_ele):
    W_gradient = get_W_gradient(A, X, y, num_ele)
    B_gradient = get_B_gradient(A, y, num_ele)

    W = W - (W_gradient.T * learning_rate)
    B = B - (B_gradient * learning_rate)

    return W, B
    

    

In [21]:
def gradient_descent(X, y , learning_rate, iterations):
    W = np.zeros((len(X),1))
    B = 0
    num_ele = len(X[0])
    for i in range(iterations):
        A = sigmoid((W.T).dot(X)+ B)
        W, B = get_gradient(A, X, y, W, B, learning_rate, num_ele)

    return W, B

In [22]:
#Create a new column to see if there are more than three fandago_stars
df['three_stars'] = df['Fandango_Stars'].apply(lambda x: 0 if x <= 3.0 else 1)
new_df = df[['Fandango_Ratingvalue', 'Fandango_Difference']]
X = np.array(new_df)
y = np.array(df['three_stars'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train.T
Y_train = y_train.reshape(1, X_train.shape[1])
X_test = X_test.T
Y_test = y_test.reshape(1, X_test.shape[1])

#Set learning rate and iterations
learning_rate = 0.01
iterations = 100

#Find best W and B for Y = WX + B / Y = ax + b
W , B = gradient_descent(X_train, y_train, learning_rate, iterations )


In [23]:
def accuracy(X, y, W, B):
    predictions = sigmoid((W.T).dot(X)+ B)
    predictions = predictions > 0.5
    predictions = np.array(predictions, dtype = 'int64').flatten()
    print("Accuracy (%):",metrics.accuracy_score(y_test, predictions)*100,"%")


In [24]:
accuracy(X_test, y_test, W, B)

Accuracy (%): 88.63636363636364 %


Sources that helped me:   
https://www.youtube.com/watch?v=sDv4f4s2SB8  
https://www.youtube.com/watch?v=U1omz0B9FTw&list=PLuhqtP7jdD8Chy7QIo5U0zzKP8-emLdny&index=1   
https://www.youtube.com/watch?v=ar8mUO3d05w&list=PLuhqtP7jdD8Chy7QIo5U0zzKP8-emLdny&index=2
https://www.youtube.com/watch?v=0VMK18nphpg&list=PLuhqtP7jdD8Chy7QIo5U0zzKP8-emLdny&index=3   
https://www.youtube.com/watch?v=t6MVuMavbBY&list=PLuhqtP7jdD8Chy7QIo5U0zzKP8-emLdny&index=4 
https://www.youtube.com/watch?v=nzNp05AyBM8&list=PLuhqtP7jdD8Chy7QIo5U0zzKP8-emLdny&index=6   
https://github.com/Jaimin09/Coding-Lane-Assets/tree/main/Logistic%20Regression%20in%20Python%20from%20Scratch   
https://zhuanlan.zhihu.com/p/28408516   

